# Introduction

In [1]:
from ipywebcam import WebCamWidget, WebCamRecorder, Record, RecordPlayer, FileListFactory

In [2]:
factory = FileListFactory("test", "$i4-$Y-$m-$d-$H-${M}-${S}-$f-$uh6.mp4", condition=FileListFactory.create_frame_based_condition(100))

In [3]:
w = WebCamWidget(
    iceServers=[{"urls": "turn:190.92.221.222", "username": "admin", "credential": "123456"}],
    constraints={
        "video": {
            "frameRate": { "max": 10 },
            "width": 256,
            "height": 192,
        },
    },
)
w

WebCamWidget(constraints={'video': {'frameRate': {'max': 10}, 'width': 256, 'height': 192}}, iceServers=[{'url…

In [4]:
recorder = WebCamRecorder(w, factory)
recorder.start()

load 6 records


In [5]:
import cv2
from av import VideoFrame
async def transform(frame, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

transformer = w.add_video_transformer(transform)

In [6]:
w.remove_video_transformer(transformer)

In [7]:
recorder.stop()

In [8]:
w.close_peers()

In [19]:
recorder = WebCamRecorder(None, factory)
recorder.factory.load()
print(recorder.factory.record_count())
player = RecordPlayer(recorder=recorder)
player.loop = True
def fix_time(frame, record, context):
    if "inited" not in context:
        context["inited"] = True
        if frame.pts > 0:
            offset = context["offset"] = frame.pts
        else:
            offset = 0
    else:
        offset = context.get("offset")
        offset = 0 if offset is None else offset
    if offset > 0:
        pass
        frame.pts -= offset
    return frame
#player.add_video_transformer(fix_time)

async def transform(frame, r, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

player.add_video_transformer(fix_time, 'test')

load 8 records
8


In [20]:
player

RecordPlayer(loop=True)

In [21]:
getattr(player, '__channel_transformers')

AttributeError: 'RecordPlayer' object has no attribute '__channel_transformers'

In [1]:
import av
from io import BytesIO
out = BytesIO()
def print_frame(frame, tag="frame"):
    print(f"[{tag}] index: {frame.index:02}, key_frame: {frame.key_frame}, dts: {frame.dts}, pts: {frame.pts}, time_base: {frame.time_base}, time: {frame.time:.2f}, pict_type: {frame.pict_type}")
def print_packet(packet, tag="packet"):
    print(f"[{tag}] stream_idx: {packet.stream_index}, dts: {packet.dts}, pts: {packet.pts}, time_base: {packet.time_base}, duration: {packet.duration}, is_keyframe: {packet.is_keyframe}")
    
with av.open("0001-2023-02-15-18-14-29-925392-29146c.mp4", "r") as input:
    in_stream = input.streams.video[0]
    last_frame = None
    for frame in input.decode(in_stream):
        if last_frame is not None:
            if frame.pts <= last_frame.pts:
                print("here")
            
        print_frame(frame)
        last_frame = frame
print("------------")
with av.open("0001-2023-02-15-18-14-29-925392-29146c.mp4", "r") as input:
    with av.open(out, "w", format="mp4") as output:
        in_stream = input.streams.video[0]
        print(f"[in_stream]: width: {in_stream.width}, height: {in_stream.height}, pix_fmt: {in_stream.pix_fmt}, base_rate: {in_stream.base_rate}, average_rate: {in_stream.average_rate}, guessed_rate: {in_stream.guessed_rate}, duration: {in_stream.duration}")
        codec_name = in_stream.codec_context.name  # Get the codec name from the input video stream.
        fps = in_stream.codec_context.rate or in_stream.rate  # Get the framerate from the input video stream.
        print(f"{codec_name} / {fps}")
        out_stream = output.add_stream(codec_name, in_stream.base_rate or in_stream.guessed_rate or in_stream.average_rate or 30)
        out_stream.width = in_stream.codec_context.width  # Set frame width to be the same as the width of the input stream
        out_stream.height = in_stream.codec_context.height  # Set frame height to be the same as the height of the input stream
        out_stream.pix_fmt = in_stream.codec_context.pix_fmt  # Copy pixel format from input stream to output stream
        print(f"[out_stream]: width: {out_stream.width}, height: {out_stream.height}, pix_fmt: {out_stream.pix_fmt}, base_rate: {out_stream.base_rate}, average_rate: {out_stream.average_rate}, guessed_rate: {out_stream.guessed_rate}, duration: {out_stream.duration}")
        inited = False
        for frame in input.decode(in_stream):
            if not inited:
                inited = True
                offset = frame.pts
            # frame.pts -= offset
            print_frame(frame)
            for out_packet in out_stream.encode(frame):
                print_packet(out_packet)
                output.mux(out_packet)
            
        for packet in out_stream.encode(None):
            output.mux(packet)
           
print("after")
out.seek(0)
with av.open(out, "r") as input:
    for frame in input.decode():
        print_frame(frame)

[frame] index: 00, key_frame: 1, dts: 70656, pts: 70656, time_base: 1/15360, time: 4.60, pict_type: I
[frame] index: 01, key_frame: 0, dts: 72704, pts: 72704, time_base: 1/15360, time: 4.73, pict_type: P
[frame] index: 02, key_frame: 0, dts: 73216, pts: 73216, time_base: 1/15360, time: 4.77, pict_type: B
[frame] index: 03, key_frame: 0, dts: 75776, pts: 75776, time_base: 1/15360, time: 4.93, pict_type: B
[frame] index: 04, key_frame: 0, dts: 76800, pts: 76800, time_base: 1/15360, time: 5.00, pict_type: B
[frame] index: 05, key_frame: 0, dts: 78848, pts: 78848, time_base: 1/15360, time: 5.13, pict_type: P
[frame] index: 06, key_frame: 0, dts: 79360, pts: 79360, time_base: 1/15360, time: 5.17, pict_type: B
[frame] index: 07, key_frame: 0, dts: 81920, pts: 81920, time_base: 1/15360, time: 5.33, pict_type: B
[frame] index: 08, key_frame: 0, dts: 82944, pts: 82944, time_base: 1/15360, time: 5.40, pict_type: B
[frame] index: 09, key_frame: 0, dts: 84992, pts: 84992, time_base: 1/15360, time:

C:\Users\cxj\AppData\Local\Temp\ipykernel_4016\3155405358.py:25: AVDeprecationWarning: VideoStream.rate is deprecated as it is not always set; please use VideoStream.average_rate.
  fps = in_stream.codec_context.rate or in_stream.rate  # Get the framerate from the input video stream.


[packet] stream_idx: 0, dts: 208, pts: 208, time_base: 1/30, duration: 0, is_keyframe: False
[frame] index: 67, key_frame: 0, dts: 174080, pts: 174080, time_base: 1/15360, time: 11.33, pict_type: B
[packet] stream_idx: 0, dts: 210, pts: 214, time_base: 1/30, duration: 0, is_keyframe: False
[frame] index: 68, key_frame: 0, dts: 176128, pts: 176128, time_base: 1/15360, time: 11.47, pict_type: P
[packet] stream_idx: 0, dts: 214, pts: 227, time_base: 1/30, duration: 0, is_keyframe: False
[frame] index: 69, key_frame: 0, dts: 177152, pts: 177152, time_base: 1/15360, time: 11.53, pict_type: B
[packet] stream_idx: 0, dts: 215, pts: 222, time_base: 1/30, duration: 0, is_keyframe: False
[frame] index: 70, key_frame: 0, dts: 179200, pts: 179200, time_base: 1/15360, time: 11.67, pict_type: B
[packet] stream_idx: 0, dts: 220, pts: 220, time_base: 1/30, duration: 0, is_keyframe: False
[frame] index: 71, key_frame: 0, dts: 180224, pts: 180224, time_base: 1/15360, time: 11.73, pict_type: B
[packet] s